<a href="https://colab.research.google.com/github/JunHyeong-data/ML-DL-Study/blob/main/Basic-Deep-Learning/17_%EB%94%A5%EB%9F%AC%EB%8B%9D%2C_%ED%92%80%EB%A7%81_%EB%A0%88%EC%9D%B4%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pooling Layer와 Modern CNN에서의 Spatial Dimension Reduction

## 1. Pooling Layer란?

고전적인 CNN 아키텍처에서 자주 사용되던 레이어로,  
**Spatial Dimension(가로·세로 크기)을 줄이는 역할**을 한다.

대표적인 풀링 방식은 다음과 같다.

- **Max Pooling**
- **Average Pooling**

현대적인 CNN에서는 사용 빈도가 줄었지만,  
공간 차원을 줄인다는 개념 자체는 매우 중요하다.

---

## 2. Max Pooling의 동작 원리

### 예제 1: 간단한 크기 변화
- 입력 텐서 크기: `4 × 4`
- 커널 사이즈: `2 × 2`
- 출력 텐서 크기: `2 × 2`

즉, 가로·세로가 각각 절반으로 줄어든다.

### 예제 2: 값 선택 방식
각 `2 × 2` 영역에서 **최댓값만 선택**하여 출력한다.

예를 들어, 입력 값이 다음과 같이 주어졌을 때:

```

1 3 | 5 7
2 4 | 6 8
---------

1 7 | 3 9
2 5 | 4 6

```

Max Pooling 결과는 각 영역의 최대값만 남는다.

---

## 3. Average Pooling

Max Pooling 대신  
각 영역의 **평균값을 출력**하는 방식이다.

- 정보 손실은 여전히 존재
- Max Pooling보다 부드러운 결과를 만든다

---

## 4. Spatial Dimension 감소 효과

- 입력 이미지 크기: `512 × 512`
- Kernel size = 2인 Pooling 적용 시:
  - 출력 크기: `256 × 256`
  - 면적 기준으로는 **1/4 감소**

Pooling을 여러 번 반복하면:
```

512 → 256 → 128 → 64 → 32

```

아주 빠르게 Spatial Dimension이 줄어든다.

---

## 5. Pooling Layer의 한계

모던 CNN에서 잘 사용되지 않는 이유는 다음과 같다.

1. **정보 손실이 큼**
   - Max Pooling은 선택된 값 외의 정보가 완전히 사라짐

2. **학습 가능한 파라미터가 없음**
   - Convolution Layer는 weight를 학습
   - Pooling Layer는 고정 연산

이 때문에 중요한 feature가 버려질 가능성이 있다.

---

## 6. Modern CNN의 대안: Strided Convolution

현대 CNN에서는 Pooling 대신  
**Convolution Layer의 stride 옵션**을 사용한다.

### 장점
- Spatial Dimension 감소
- 동시에 **학습 가능한 weight 유지**
- 정보 손실 최소화

---

## 7. Global Average Pooling (GAP)

### 개념
- Feature Extraction의 **마지막 텐서에서**
- **각 채널별로 평균을 계산**

### 예시
- 마지막 Feature Tensor 크기: `128 × 16 × 16`
  - 채널 수: 128
  - 각 채널의 `16 × 16` 값을 평균

결과:
- `128`차원의 벡터 생성

---

## 8. 기존 방식 vs Global Average Pooling

### 기존 CNN 구조
1. Feature Map (`128 × 16 × 16`)
2. Flatten → 약 32,000차원 벡터
3. Fully Connected Layer
4. Classification

➡ 연산량과 파라미터 수가 매우 큼

### Global Average Pooling 적용
1. Feature Map (`128 × 16 × 16`)
2. Channel-wise Average
3. `128`차원 벡터
4. 바로 Classification

➡ **연산량 대폭 감소**

---

## 9. Global Average Pooling의 장점

1. **연산량 감소**
2. **과적합 위험 감소**
3. **위치 정보 제거**
   - Classification 문제에서는 객체 위치가 중요하지 않음
   - 위치에 무관한 robust한 분류 가능

---

## 10. 정리

- CNN은 크게 두 부분으로 나뉜다
  - Feature Extraction
  - Classification

- Pooling Layer
  - Spatial Dimension을 빠르게 줄임
  - 하지만 정보 손실 큼
  - 현대 CNN에서는 거의 사용되지 않음

- Modern CNN
  - Strided Convolution으로 공간 축소
  - Global Average Pooling으로 FC Layer 대체

---

## 다음 내용
다음 시간에는 **Normalization Layer**에 대해 다룬다.

In [2]:
import torch
import torch.nn as nn

input_tensor = torch.Tensor([[[[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12],
                             [13, 14, 15, 16]]]])
print(input_tensor.shape)
maxpool_layer = nn.MaxPool2d(kernel_size=2)
output_tensor = maxpool_layer(input_tensor)
print(output_tensor)

torch.Size([1, 1, 4, 4])
tensor([[[[ 6.,  8.],
          [14., 16.]]]])


In [3]:
avgpool_layer = nn.AvgPool2d(kernel_size=2)
output_tensor = avgpool_layer(input_tensor)
print(output_tensor)

tensor([[[[ 3.5000,  5.5000],
          [11.5000, 13.5000]]]])


In [5]:
import torch
import torchvision
import matplotlib.pyplot as plt

my_device = torch.device('cpu')
print(my_device)

cpu


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
class ModerGAPCNN(nn.Module):
  def __init__(self, num_classes=10):
    super().__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
        nn.ReLU(inplace=True),
    )
    self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.features(x)
    x = self.global_avg_pool(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

net = ModerGAPCNN(num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

| 항목          | 이유                |
| ----------- | ----------------- |
| 공간 크기 ↓     | 계산량 감소, 위치 불변성 증가 |
| 채널 수 ↑      | 더 많은 특징 표현 가능     |
| stride Conv | 학습 가능한 다운샘플링      |


In [10]:
net.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for batch_idx, (data, label) in enumerate(trainloader):
        data, label = data.to(my_device), label.to(my_device)
        scores = net(data)
        loss = criterion(scores, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    net.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(my_device), label.to(my_device)
            scores = net(data)
            loss = criterion(scores, label)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(label).sum().item()

    val_loss /= len(testloader.dataset)
    val_accuracy = 100. * correct / len(testloader.dataset)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch [1/100], Training Loss: 1.4457, Validation Loss: 1.5384, Validation Accuracy: 45.64%
Epoch [2/100], Training Loss: 1.2725, Validation Loss: 1.4052, Validation Accuracy: 49.48%
Epoch [3/100], Training Loss: 1.2714, Validation Loss: 1.3259, Validation Accuracy: 53.03%
Epoch [4/100], Training Loss: 1.4004, Validation Loss: 1.2737, Validation Accuracy: 54.40%


KeyboardInterrupt: 